# JEM092 Asset Pricing - Homework 2

### Group 82780095

Petr Dvořáček, Jan Kubal, Matyáš Mattanelli

---

## Abstract

## 1. Introduction

## 2. Data Description

In this section we provide an overview of the utilized data. We examine summary staistics of the variables as well as pairwise correlations. Lastly, we perform a persistence analysis. Firstly, we load the data along with the necessary packages.

In [1]:
#Loading the packages
#install.packages("xts", "readr", "lubridate", "quantmod", "moments", "sandwich", "lmtest")
suppressPackageStartupMessages({
    library(xts) #Handling xts objects
    library(readr) #Reading csv files
    library(lubridate) #Date manipulation
    library(quantmod) #Returns calculation
    library(moments) #Skewness and kurtosis functions
    library(sandwich) #Robust covariance matrix estimation
    library(lmtest) #Robust covariance matrix estimation
})

#Loading the data
monthly_returns <- readRDS("Final data/monthly_excess_returns.RData")
skewness_data <- readRDS("Final data/skewness.RData")
betas <- readRDS("Final data/betas.RData")
market_cap <- readRDS("Final data/market_cap_monthly.RData")
size <- readRDS("Final data/size.RData")
fffactors <- readRDS("Final data/fffactors_monthly.RData")
momentum <- readRDS("Final data/momentum.RData")
liquidity <- readRDS("Final data/liquidity.RData")

Before proceeding to the summary statistics, we need to take a look at the inspected period. The required time span according to the first homework is 01.01.2007 - 28.02.2022. However, in the second assignment the required period is 01.01.2007 - 31.12.2020. We will select the former in order to utilize as much data as possible. In addition, the data for the Market Capitalization variable are available from 28.02.2009 onwards which decreases the usable time span. Furthermore, the liquidity measure ends in December 2021 which shrinks our data even further. Nevertheless, for the purpose of data description, we retain all values for all variables available after 01.01.2007 and until 28.02.2022. We will make the proper adjustments later in the empirical analysis.

In [2]:
#Restricting the period (01/01/2007 - 28/02/2022)
market_cap <- market_cap["/2022-02-28"]
size <- size["/2022-02-28"]
fffactors <- fffactors["/2022-02-28"]
momentum <- momentum["/2022-02-28"]

Finally, we can proceed with describing the data. Our final data set contains monthly observations for 250 stocks in the period between 01.01.2077 and 28.02.2022. The three sorting variables that we inspect are Beta, Size, and Skewness. In addition, the dependent variable in our setting are the one-ahead excess returns. We also utilize Market Capitalization for value-weighted portfolios, and therefore we provide the statistics for this variable as well. For each variable and each period we calcute 12 statistics. We then take an average of these values over all available periods. In order to facilitate the process, we define a custom function.

In [3]:
#Defining a function to calculate the summary statistics
calc_sum_stats <- function(var_data) {
  period_stats <- matrix(ncol = 12, nrow = nrow(var_data)) #A matrix for the statistics from each period
  perc_5 <- function(x, na.rm) {return(quantile(x, probs = 0.05, na.rm = na.rm))} #Function to calculate the 5th percentile of data
  perc_25 <- function(x, na.rm) {return(quantile(x, probs = 0.25, na.rm = na.rm))} #Function to calculate the 25th percentile of data
  perc_75 <- function(x, na.rm) {return(quantile(x, probs = 0.75, na.rm = na.rm))} #Function to calculate the 75th percentile of data
  perc_95 <- function(x, na.rm) {return(quantile(x, probs = 0.95, na.rm = na.rm))} #Function to calculate the 25th percentile of data
  calc_n <- function(x, na.rm) {return(sum(!is.na(x)))} #Function to calculate the number of observations
  funcs <- list(mean, sd, skewness, kurtosis, min, perc_5, perc_25, median, perc_75, perc_95, max, calc_n) #Store the functions we will use
  for (i in 1:nrow(var_data)) { #Loop through the periods
    for (j in 1:length(funcs)) { #Loop through the functions
      period_stats[i, j] <- funcs[[j]](as.numeric(var_data[i, ]), na.rm = T) #Apply each function and store the result 
    }
  }
  time_avg <- apply(period_stats, 2, mean, na.rm = T) #Take a time series mean of each statistic
  return(round(time_avg, 2)) #Round to two decimals and return the result
}

#Applying the function
sum_stats <- as.data.frame(matrix(nrow = 5, ncol = 12)) #Place holder for the final summary statistics
colnames(sum_stats) <- c("Mean", "Sd", "Skewness", "Kurtosis", "Min", "5%", "25%", "Median", "75%", "95%", "Max", "n") #Rename the columns as the statistics
var_names <- c("Returns", "Skewness", "Beta", "Size", "Market cap") #Define a vector of variable names
row.names(sum_stats) <- var_names #Rename the row names as the variables
data_list <- list(monthly_returns, skewness_data, betas, size, market_cap) #Store the data in a list for looping
for (i in 1:5) { #Loop through the variables and apply the function
  sum_stats[i, ] <- calc_sum_stats(data_list[[i]])
}

The resulting summary statistics are available below. Each row shows the 12 statistics for each of the 5 variables. A brief glance at the first row reveals that the average excess return in our data set is negative. Both the mean and median are below zero and close to each other suggesting a symetric distribution. This is confirmed by the skewness (first row, third column) which is quite close to zero. The value for kurtosis on the other hand is well above 3 (kurtosis of normal distribution) and therefore, the distribution of the returns is likely heavy-tailed. The maximum and minimum excess returns are 29% and -29%, respectively. The maximum value, however, is likely an outlier given the relatively low value of the 95th percentile. 

As for the Skewness sorting variable, it seems to be distributed symmetrically around zero. This confirms our previous observations that the returns are symmetric. It doesa attain some extreme values on both tails.

A second sorting variable of interest is the stocks' sensitivity to market. As we can see, the values are on average very low and distributed densely around zero. In addition, it does not attain a single negative value suggesting that all the stocks react positively to the market. 

The Market Capitalization seems to be very far from normal distribution, however, the logarithmic transformation seems to remedy this issue since the Size variable is quite close to normal distribution in temrs of both skewness and kurtosis.

Overall, we can see that in the average period we have majority of the observations for all stocks (denoted by the last column) with the exception of the Market Capitalization and Size variables which seem to have slightly more missing observations. 

In [4]:
#Summary statistics
sum_stats

,Mean,Sd,Skewness,Kurtosis,Min,5%,25%,Median,75%,95%,Max,n
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Returns,-0.05,0.07,0.50,7.39,-0.29,-0.16,-0.10,-0.06,-0.02,0.06,0.29,248.78
Skewness,0.02,0.97,0.26,18.38,-4.89,-1.20,-0.34,-0.02,0.34,1.45,5.41,248.64
Beta,0.01,0.00,0.39,3.34,0.00,0.00,0.01,0.01,0.01,0.02,0.02,248.64
Size,2.99,1.12,0.60,3.24,0.43,1.47,2.18,2.84,3.63,5.08,6.56,244.67
Market cap,45.48,90.88,5.21,38.28,1.82,5.06,9.77,18.46,40.71,168.12,858.28,245.14


Moving on from the summary statistic, we provide the average pairwise correlations between the variables in question. Similarly as in the previous case, we firstly calculate the cross-sectional correlations between the variables and the average the values across periods.

In [5]:
#Defining a function to calculate the average correlation between two variables
calc_corr <- function(var1, var2, method = "pearson") { #Expects data for both variables and an indicator for type (default is pearson)
  var1 <- var1[index(var1) %in% index(var2)] #Filter only data available for both variables
  cross_sec_corrs <- rep(NA, nrow(var1)) #Empty vector the results of each period
  for (i in 1:nrow(var1)) { #Loop through the periods available for the first variable
    cross_sec_corrs[i] <- cor(as.numeric(var1[i]), as.numeric(var2[index(var1)[i]]), use = "pairwise.complete.obs", method = method) #Calculate the correlation for each period
  }
  return(round(mean(cross_sec_corrs, na.rm = T), 2)) #Return the rounded time series average of the correlations
}

#Applying the function
corrs <- as.data.frame(matrix(nrow = 5, ncol = 5)) #Empty matrix for the results
colnames(corrs) <- var_names #Column names
row.names(corrs) <- var_names #Row names
indices <- combn(1:5, 2) #Get the indices for each pair (unique and excluding pairing variables with themselves)
for (i in 1:ncol(indices)) { #Loop through the pairs
  corrs[indices[1, i], indices[2, i]] <- calc_corr(data_list[[indices[1, i]]], data_list[[indices[2, i]]], method = "spearman") #Spearman correlation in the upper triangle
  corrs[indices[2, i], indices[1, i]] <- calc_corr(data_list[[indices[2, i]]], data_list[[indices[1, i]]]) #Pearson correlation in the lower triangle
}

The results are available in the table below. The diagonal contains missing values since the correlation of a variable with itself is 1 and as such is not relevant. Above the diagonal we present the Spearman correlation while below is the Pearson correlation. As can be seen, the values are largely similar with the exception of Size and Market Capitalization since the Spearman correlation captures the perfectly monotonic relationship while Pearson correlation shows only the degree of linear dependence. As for the remaining values, we can see that the variable are mostly uncorrelated. Nevertheless, the Skewness measure seems to be slightly positively correlated with the Returns and with Beta. In addition, Size and Beta appear to be negatively correlated suggesting that larger stocks are less market-sensitive.

In [6]:
#Pairwise correlations
corrs

,Returns,Skewness,Beta,Size,Market cap
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Returns,NA,0.10,0.02,-0.01,-0.01
Skewness,0.13,NA,0.15,0.01,0.01
Beta,0.03,0.12,NA,-0.13,-0.13
Size,-0.01,0.02,-0.14,NA,1.00
Market cap,0.00,0.03,-0.08,0.75,NA


Lastly, we perform the persistence analysis. We consider five lags and present the average autocorrelation over all periods for a given lag for each variable.

In [7]:
#Defining a function to calculate persistence of a variable
calc_pers <- function(var_data, lags = 5) { #Expects an xts object and the number of lags how far to calculate the correlations
  final_persist <- rep(NA, lags) #A vector to store the results 
  for (i in 1:lags) { #Loop through the lags
    cross_persist <- rep(NA, nrow(var_data)) #A vector to store the results from each period
    for (j in 1 :nrow(var_data)) { #Loop through the periods
      current_period <- index(var_data)[j] #Store the current period
      lagged_period <- as.Date(as.yearmon(current_period %m-% months(i)), frac = 1) #Derive the lagged period
      if (!lagged_period %in% index(var_data)) { #If the lag is unavailable, skip
        next
      }
      cross_persist[j] <- cor(as.numeric(var_data[j]), as.numeric(var_data[lagged_period]), use = "pairwise.complete.obs") #Calculate Pearson between the variable and the appropriate lag
    }
    final_persist[i] <- mean(cross_persist, na.rm = T) #Calculate the time series mean
  }
  return(round(final_persist, 3)) #Return the rounded correlations
}

#Apply the function
lags <- 5 #Specify the required number of lags
persistence <- as.data.frame(matrix(ncol = lags, nrow = 5)) #Data frame for the results (lags in columns, vars in rows)
colnames(persistence) <- paste("Lag ", 1:lags) #Rename the columns with lag numbers
row.names(persistence) <- var_names #Rename the rows with variable names
for (i in 1:nrow(persistence)) { #Loop through the variables and calculate their persistence
  persistence[i, ] <- calc_pers(data_list[[i]], lags = lags) #Apply the function
}

The results of the table below seem agree with basic intuition. The Returns do not show any significant autocorrelations which demostrates their unpredictable behavior. On the other hand, the remaining variables seem to be quite persitent. In case of Skewness, the persitance is slowly decaying which is likely due to the fact that lags that are close to each other contain larger portions of the same data for calculation. We cannot observe any strong decay for Beta, Size, and Market Cap which suggests that the market sensitivity of a stock does not change much over time on average and the same applies to its market share. 

In [8]:
#Persistence analysis
persistence

,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Returns,-0.033,0.020,-0.016,0.004,-0.010
Skewness,0.906,0.801,0.721,0.643,0.560
Beta,0.988,0.972,0.951,0.939,0.920
Size,0.998,0.996,0.994,0.992,0.990
Market cap,0.998,0.996,0.994,0.992,0.989


## 3. Methodology

In [9]:
#Defining a function to perform a univariate sort
univariate_sort <- function(sort_variable, no_of_ports = 5, weighted = F) {
  sort_variable_name <- gsub("MDLZ.", "", colnames(sort_variable)[1]) #Extracting the name of the sorting variable
  final_ouptut <- as.data.frame(matrix(ncol = no_of_ports + 1, nrow = 5)) #Data frame for the final output
  colnames(final_ouptut) <- c(1:no_of_ports, paste(no_of_ports, 1, sep = "-")) #Column names for clarity
  row.names(final_ouptut) <- c(paste0("Mean of ", sort_variable_name), "Mean of one-ahead returns", "T-statistic", "Alpha", "T-statistic (alpha)")
  all_avg_sort_values <- c() #Empty vector to which we will append the results from each period (sort variable)
  all_avg_returns <- c() #Empty vector to which we will append the results from each period (returns)
  for (i in 1:nrow(sort_variable)) { #Loop through the rows (periods)
    current_period <- index(sort_variable)[i] #Store the current period
    next_period <- as.Date(as.yearmon(current_period %m+% months(1)), frac = 1) #Store the next period (for the 1-ahead returns)
    if (!next_period %in% index(monthly_returns)) { #In case we do not have 1-ahead returns, skip the period
      next
    }
    average_sort_values <- rep(NA, no_of_ports) #Place holder for the cross-sectional average values of the sort variable
    average_returns <- xts(matrix(nrow = 1, ncol = no_of_ports + 1), order.by = current_period) #Place holder for the cross-sectional average values of the 1-ahead returns
    breakpoints <- quantile(sort_variable[current_period], probs = seq(1/no_of_ports, 1 - 1/no_of_ports, 1/no_of_ports), na.rm = T) #Find the breakpoints
    for (j in 1:no_of_ports) { #Looping through the portfolios
      if (j == 1) { #For the first portfolio we have only one condition
        indic <- which(sort_variable[current_period] <= breakpoints[j]) #Find the stocks in the portfolio and save their column indices
      } else if (j == no_of_ports) { #For the last portfolio we have only one condition as well
        indic <- which(sort_variable[current_period] >= breakpoints[j -  1])
      } else { #The rest of the portfolios (two conditions)
        indic <- which(sort_variable[current_period] >= breakpoints[j - 1] & sort_variable[current_period] <= breakpoints[j]) #"=" at both inequalities to prevent empty portfolios
      }
      if (weighted == T) { #Market cap weighted average
        mc_weights <- as.numeric(market_cap[current_period, indic]) #Storing the weights
        if (length(mc_weights) == 0) { #If we do not have the market cap data, we have to skip
          next
        }
        mc_weights[is.na(mc_weights)] <- 0 #0 value for missings = they have no weight
        average_returns[, j] <- weighted.mean(as.numeric(monthly_returns[next_period, indic]), w = mc_weights, na.rm = T)
      } else { #Equally-weighted average
        average_returns[, j] <- mean(monthly_returns[next_period, indic], na.rm = T) #Calculate the average of 1-ahead returns
      }
      average_sort_values[j] <- mean(sort_variable[current_period, indic], na.rm = T) #Calculate the average of the sort variable for the given portfolio
    }
    average_returns[, no_of_ports + 1] <- average_returns[, no_of_ports] - average_returns[, 1] #The difference portfolio
    all_avg_sort_values <- rbind(all_avg_sort_values, average_sort_values) #Append the results
    all_avg_returns <- rbind(all_avg_returns, average_returns) #Append the results
  }
  final_ouptut[1, ] <- c(apply(all_avg_sort_values, 2, mean, na.rm = T) , NA) #Final average values of the sort variable
  for (i in 1:(no_of_ports + 1)) { #Loop through the portfolios and get the time series avg of returns, t-stat and alpha
    model_avg <- lm(na.omit(all_avg_returns[, i]) ~ 1) #Regress returns of each portfolio on the intercept (for weighted portfolio there may be missing values and coeftest cannot handle them => remove)
    model_avg_adj <- coeftest(model_avg, vcov = NeweyWest(model_avg, lag = 6)) #Make Newey-West adjustment of std errors
    final_ouptut[2, i] <- model_avg_adj[1, 1] #Store the intercept (=time series average)
    final_ouptut[3, i] <- model_avg_adj[1, 3] #Store the t-statistic
    #Regress the returns on the five factors
    factor_data <- na.omit(merge.xts(all_avg_returns[, i], fffactors[, 1:3], momentum, liquidity)) #Merge the data needed for the factor model (to secure the same number of obs) + remove NAs since coeftest cannot handle it
    factor_model <- lm(factor_data[, 1] ~ factor_data[, 2] + factor_data[, 3] + factor_data[, 4] + factor_data[, 5] + 
                         factor_data[, 6])
    factor_model_adj <- coeftest(factor_model, vcov = NeweyWest(factor_model, lag = 6)) #Make Newey-West adjustment
    final_ouptut[4, i] <- factor_model_adj[1, 1] #Store the intercept (alpha)
    final_ouptut[5, i] <- factor_model_adj[1, 3] #Store the t-statistic
  }
  return(final_ouptut)
}

In [10]:
#Defining a function to perform a bivariate sort (independent and weighted)
bivariate_sort <- function(sort_variable1, sort_variable2, percs1 = c(0.3, 0.7), percs2 = c(0.3, 0.7)) { #Accepts two sort variables data and required percentiles for the portfolios
  no_of_ports1 <- length(percs1) + 1 #Storing the number of portfolios of the first sort variable
  no_of_ports2 <- length(percs2) + 1 #Storing the number of portfolios of the second sort variable
  sort_variable1_name <- gsub("MDLZ.", "", colnames(sort_variable1)[1]) #Extracting the name of the first sorting variable
  sort_variable2_name <- gsub("MDLZ.", "", colnames(sort_variable2)[1]) #Extracting the name of the second sorting variable
  final_ouptut <- as.data.frame(matrix(ncol = no_of_ports1 + 1, nrow = 4*(no_of_ports2 + 1))) #Data frame for the final output
  colnames(final_ouptut) <- c(paste0(sort_variable1_name, " ", 1:no_of_ports1), paste0(sort_variable1_name, " ", paste0(no_of_ports1, "-", 1))) #Column names for clarity
  sort_variable2_ports <- c(paste0(sort_variable2_name, " ", 1:no_of_ports2), paste0(sort_variable2_name, " ", paste0(no_of_ports2, "-", 1))) #Portfolios of the second sort variable stored to use for row names of the final output
  statistics <- c("Mean", "T-statistic", "Alpha", "T-statistic (alpha)") #Statistics to paste for row names
  row.names(final_ouptut) <- as.vector(t(outer(sort_variable2_ports, statistics, FUN = "paste", sep = " - "))) #Row names for clarity
  all_avg_returns <- list() #Empty vector to which we will append the matrices of results from each period (returns)
  find_indics <- function(xts_object, percs) { #A function that returns boolean vectors for given quantiles of given data
    breakpoints <- quantile(xts_object, probs = percs, na.rm = T) #Find the breakpoints
    no_of_quants <- length(breakpoints) + 1 #Store the number of quantiles
    indics <- vector("list", no_of_quants) #Empty list to store the final indices
    for (i in 1:no_of_quants) { #Loop through the quantiles
      if (i == 1) { #For the first quantile we have only one condition
        indic <- xts_object <= breakpoints[i] #Find the stocks in the first quantile
      } else if (i == no_of_quants) { #For the last quantile we have only one condition as well
        indic <- xts_object >= breakpoints[i -  1]
      } else { #The rest of the quantiles (two conditions)
        indic <- xts_object >= breakpoints[i - 1] & xts_object <= breakpoints[i] #"=" at both inequalities to prevent empty quantiles (see book)
      }
      indics[[i]] <- indic
    }
    return(indics)
  }
  for (i in 1:nrow(sort_variable1)) { #Loop through the rows (periods) of the first sorting variable (we are interested in the intersection of the periods so we can loop through just one of the indices)
    current_period <- index(sort_variable1)[i] #Store the current period
    next_period <- as.Date(as.yearmon(current_period %m+% months(1)), frac = 1) #Store the next period (for the 1-ahead returns)
    if (!next_period %in% index(monthly_returns) | !current_period %in% index(sort_variable2)) { #In case we do not have 1-ahead returns or the values for the second variable, skip the period
      next
    }
    average_returns <- matrix(nrow = no_of_ports2 + 1, ncol = no_of_ports1 + 1) #Place holder for the cross-sectional average values of the 1-ahead returns
    indics1 <- find_indics(sort_variable1[current_period], percs1) #Divide the first sorting variable into quantiles (get the boolean values for the stocks)
    indics2 <- find_indics(sort_variable2[current_period], percs2) #Divide the second sorting variable into quantiles (get the boolean values for the stocks)
    indics_comb <- expand.grid(indics1, indics2) #Make all possible combinations for all quantiles of both variables
    indics_for_mat <- expand.grid(1:no_of_ports1, 1:no_of_ports2) #Indices for writing the returns in the matrix for each period
    for (j in 1:nrow(indics_comb)) { #Loop through the portfolios
      final_indics <- which(unlist(indics_comb[j, 1]) & unlist(indics_comb[j, 2])) #Find the column indices for a given portfolio
      mc_weights <- as.numeric(market_cap[current_period, final_indics]) #Storing the market cap weights
      if (length(mc_weights) == 0) { #If we do not have the market cap data, we have to skip
        next
      }
      mc_weights[is.na(mc_weights)] <- 0 #0 value for missings = they have no weight
      average_returns[indics_for_mat[j, 2], indics_for_mat[j, 1]] <- weighted.mean(as.numeric(monthly_returns[next_period, final_indics]), w = mc_weights, na.rm = T)  
    }
    for (j in 1:(ncol(average_returns)- 1)) { #Calculate the difference portfolios of the second sorting variable (last row)
      average_returns[nrow(average_returns), j] <- average_returns[nrow(average_returns) - 1, j] - average_returns[1, j] 
    }
    for (j in 1:nrow(average_returns)) { #Calculate the difference portfolios of the first sorting variable + the diff in diff portfolio (last column)
      average_returns[j, ncol(average_returns)] <- average_returns[j, ncol(average_returns) - 1] - average_returns[j, 1]
    }
    all_avg_returns[[length(all_avg_returns) + 1]] <- average_returns #Append the matrix with results for the current period
    names(all_avg_returns)[length(all_avg_returns)] <- as.character(current_period) #Name the element in the list with the period for later usage
  }
  indics_for_ports <- expand.grid(1:(no_of_ports1 + 1), 1:(no_of_ports2 + 1)) #Store the indices for each portfolio in the returns matrix for the loop
  final_returns <- rep(NA, ((no_of_ports1 + 1)*(no_of_ports2 + 1))) #A vector to store the final returns for each portfolio
  final_t_stats <- rep(NA, ((no_of_ports1 + 1)*(no_of_ports2 + 1))) #A vector to store the final t-statistics for each portfolio
  final_alphas <- rep(NA, ((no_of_ports1 + 1)*(no_of_ports2 + 1))) #A vector to store the final alphas for each portfolio
  final_t_stats_alphas <- rep(NA, ((no_of_ports1 + 1)*(no_of_ports2 + 1))) #A vector to store the final t-statistics for alphas for each portfolio
  for (i in 1:((no_of_ports1 + 1)*(no_of_ports2 + 1))) { #Loop through the portfolios and get the time series avg of returns, t-stat and alpha
    data_to_use <- xts(unlist(lapply(all_avg_returns, "[", indics_for_ports[i, 2], indics_for_ports[i, 1])), order.by = as.Date(names(all_avg_returns))) #Get the time series for each portfolio
    model_avg <- lm(na.omit(data_to_use) ~ 1) #Regress returns of each portfolio on the intercept (for weighted portfolio there may be missing values and coeftest cannot handle them => remove)
    model_avg_adj <- coeftest(model_avg, vcov = NeweyWest(model_avg, lag = 6)) #Make Newey-West adjustment of std errors
    final_returns[i] <- model_avg_adj[1, 1] #Store the intercept (=time series average)
    final_t_stats[i] <- model_avg_adj[1, 3] #Store the t-statistic
    #Regress the returns on the five factors
    factor_data <- na.omit(merge.xts(data_to_use, fffactors[, 1:3], momentum, liquidity)) #Merge the data needed for the factor model (to secure the same number of obs) + remove NAs since coeftest cannot handle it
    factor_model <- lm(factor_data[, 1] ~ factor_data[, 2] + factor_data[, 3] + factor_data[, 4] + factor_data[, 5] + 
                         factor_data[, 6])
    factor_model_adj <- coeftest(factor_model, vcov = NeweyWest(factor_model, lag = 6)) #Make Newey-West adjustment
    final_alphas[i] <- factor_model_adj[1, 1] #Store the intercept (alpha)
    final_t_stats_alphas[i] <- factor_model_adj[1, 3] #Store the t-statistic
  }
  final_result <- list(final_returns, final_t_stats, final_alphas, final_t_stats_alphas)
  for (i in 1:4) { #Loop through the list of results
    results_mat <- matrix(final_result[[i]], ncol = no_of_ports1 + 1, byrow = T) #Reshape the vector to matrix to facilitate filling in the final output
    final_ouptut[seq(i, nrow(final_ouptut), 4), ] <- results_mat #Fill the appropriate rows with results for each statistic
  }
  return(final_ouptut)
}

In [11]:
#Defining a function to perform the Fama-Macbeth regression
fama_macbeth <- function(indep_var) { #Expects a list of xts objects with independent variables
  cross_sec_coefs <- c() #Empty vector to which we will append the coefficients for each period
  for (i in 1:nrow(monthly_returns)) { #Loop through the periods
    next_period <- index(monthly_returns)[i] #Store the period from which the one ahead returns come
    current_period <- as.Date(as.yearmon(next_period %m-% months(1)), frac = 1) #Store the period for the independent variables
    move_to_next_period <- F #Setting up an indicator for incomplete periods
    for (j in 1:length(indep_var)) { #Check if the current period is contained in all independent variables
      if (current_period %in% index(indep_var[[j]])) { #If it is, move to next variable
        next
      } else { #If it is not, set the indicator and break
        move_to_next_period <- T
        break
      }
    }
    if (move_to_next_period == T) { #If one of the independent variables does not contain the required period, move to next period
      next
    }
    cross_sec_data <- as.data.frame(matrix(ncol = length(indep_var) + 1, nrow = ncol(monthly_returns))) #Data holder for each period
    colnames(cross_sec_data) <- c("Returns", names(indep_var)) #Column names that will be used in the formula
    cross_sec_data[, 1] <- as.numeric(monthly_returns[next_period]) #Fill in the data for the dependent variable
    for (j in 1:length(indep_var)) { #Fill in the data for each independent variable
      cross_sec_data[, j + 1] <- as.numeric(indep_var[[j]][current_period])
    }
    form <- paste("Returns", paste(colnames(cross_sec_data)[2:ncol(cross_sec_data)], collapse = "+"), sep = "~") #Define the formula
    cross_sec_model <- lm(form, data = cross_sec_data) #Estimate the model
    cross_sec_coefs <- rbind(cross_sec_coefs, cross_sec_model$coefficients) #Append the coefficients
  }
  final_output <- as.data.frame(matrix(nrow = 2, ncol = ncol(cross_sec_coefs))) #Place holder to store the final output (time series means and t-statistics)
  colnames(final_output) <- colnames(cross_sec_coefs) #Rename the columns for clarity
  row.names(final_output) <- c("Mean", "T-statistic") #Rename the rows for clarity
  for (i in 1:ncol(cross_sec_coefs)) { #Calculate the time series mean and t-statistics for each coefficient
    time_model <- lm(cross_sec_coefs[, i] ~ 1) #Regress each time series on a constant to get the mean estimate
    time_model_adj <- coeftest(time_model, vcov = NeweyWest(time_model, lag = 4)) #Newey-West adjustment of the errors
    final_output[1, i] <- time_model_adj[1, 1] #Store the mean
    final_output[2, i] <- time_model_adj[1, 3] #Store the standard error
  }
  return(final_output)
}

## 4. Results

In this section we present the results of our analyses. We start with the univariate sort.

### Univariate sort

In [12]:
#Univariate sort on betas (equally-weighted)
univariate_sort_betas <- univariate_sort(betas) #Equally weighted
univariate_sort_betas

,1,2,3,4,5,5-1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mean of Beta,0.005315965,0.008145472,0.01003064,0.01201056,0.01559522,NA
Mean of one-ahead returns,-0.040201592,-0.039654276,-0.03791159,-0.03243701,-0.02626897,0.01393262
T-statistic,-1.957493826,-1.926549270,-1.79728969,-1.49818210,-1.21802015,1.58604641
Alpha,-0.040006911,-0.041820348,-0.03961597,-0.03476045,-0.02792978,0.01207713
T-statistic (alpha),-2.210619977,-2.287397388,-2.12179514,-1.87142301,-1.52349440,1.35356534


In [13]:
#Univariate sort on betas (value-weighted)
univariate_sort_weighted_betas <- univariate_sort(betas, weighted = T) #Market cap weighted (rbind warning should be okay, just NAs)
univariate_sort_weighted_betas

Warning message in rbind(deparse.level, ...):
"mismatched types: converting objects to numeric"


,1,2,3,4,5,5-1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mean of Beta,0.005259643,0.008120456,0.01004147,0.01203002,0.01562879,NA
Mean of one-ahead returns,-0.020167871,-0.018665847,-0.01719342,-0.01333809,-0.01534456,0.0048233133
T-statistic,-0.974184860,-0.960231665,-0.82377566,-0.68878979,-0.83838204,0.6650608903
Alpha,-0.014339890,-0.016236178,-0.01567603,-0.01425867,-0.01456113,-0.0002212442
T-statistic (alpha),-0.723421375,-0.818464906,-0.79501658,-0.74252698,-0.76514972,-0.0281559398


In [14]:
#Univariate sort on size (equally weighted)
univariate_sort_size <- univariate_sort(size)
univariate_sort_size

,1,2,3,4,5,5-1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mean of Size,1.604346039,2.29910489,2.82673288,3.43900684,4.71052537,NA
Mean of one-ahead returns,-0.006697812,-0.01941954,-0.02069066,-0.02277601,-0.02293233,-0.01623452
T-statistic,-0.378612806,-1.08586516,-1.16983518,-1.34653282,-1.34493568,-6.15070719
Alpha,-0.005259066,-0.01870591,-0.01910294,-0.02181469,-0.02123653,-0.01597746
T-statistic (alpha),-0.294122017,-1.06824798,-1.04626543,-1.28623200,-1.25104668,-5.10901045


In [15]:
#Univariate sort on size (value-weighted)
univariate_sort_weighted_size <- univariate_sort(size, weighted = T) #Market cap weighted
univariate_sort_weighted_size

,1,2,3,4,5,5-1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mean of Size,1.604346039,2.29910489,2.82673288,3.43900684,4.71052537,NA
Mean of one-ahead returns,-0.008820904,-0.01978457,-0.02104353,-0.02271985,-0.02220847,-0.01338757
T-statistic,-0.500549536,-1.10688882,-1.18885446,-1.33559768,-1.32942662,-4.50971406
Alpha,-0.007017767,-0.01915427,-0.01932512,-0.02151969,-0.02101925,-0.01400149
T-statistic (alpha),-0.395582726,-1.09787702,-1.05283199,-1.25952910,-1.26499978,-3.70567811


In [16]:
#Univariate sort on skewness (equally weighted)
univariate_sort_skewness <- univariate_sort(skewness_data)
univariate_sort_skewness

,1,2,3,4,5,5-1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mean of Skewness,-1.09198069,-0.27756862,-0.02327015,0.25718155,1.21316260,NA
Mean of one-ahead returns,-0.03661514,-0.03460912,-0.03656714,-0.03511284,-0.03347537,0.003139775
T-statistic,-1.86223037,-1.64114552,-1.73617315,-1.68196818,-1.56488305,1.010771418
Alpha,-0.03698843,-0.03620937,-0.03877004,-0.03661638,-0.03548955,0.001498881
T-statistic (alpha),-2.07936273,-1.92377218,-2.15255882,-2.05716505,-1.93724474,0.574876944


In [17]:
#Univariate sort on skewness (value-weighted)
univariate_sort_weighted_skewness <- univariate_sort(skewness_data, weighted = T) #Market cap weighted (rbind warning should be okay, just NAs)
univariate_sort_weighted_skewness

Warning message in rbind(deparse.level, ...):
"mismatched types: converting objects to numeric"


,1,2,3,4,5,5-1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mean of Skewness,-1.10818631,-0.30349572,-0.04912819,0.23251124,1.19892282,NA
Mean of one-ahead returns,-0.01695480,-0.01660342,-0.01735680,-0.01789694,-0.01926040,-0.002305604
T-statistic,-0.89464714,-0.81573903,-0.86136573,-0.87563264,-1.08538556,-0.606689763
Alpha,-0.01385489,-0.01278373,-0.01621440,-0.01508792,-0.01789007,-0.004035171
T-statistic (alpha),-0.72932992,-0.66142362,-0.86618196,-0.75247336,-1.00543262,-0.938539156


### Bivariate sort

In [18]:
#Bivariate sort on size and beta
bivariate_sort_size_beta <- bivariate_sort(size, betas)
bivariate_sort_size_beta

Warning message in min(j, na.rm = TRUE):
"no non-missing arguments to min; returning Inf"
Warning message in max(j, na.rm = TRUE):
"no non-missing arguments to max; returning -Inf"


,Size 1,Size 2,Size 3,Size 3-1
,<dbl>,<dbl>,<dbl>,<dbl>
Beta 1 - Mean,-0.0139117902,-0.020302816,-0.019512761,-0.005600971
Beta 1 - T-statistic,-0.7034049538,-1.021567983,-0.957986187,-1.719324985
Beta 1 - Alpha,-0.0077536651,-0.015178535,-0.015108416,-0.007354751
Beta 1 - T-statistic (alpha),-0.4262398871,-0.796893366,-0.751317891,-2.581931780
Beta 2 - Mean,-0.0118937627,-0.017139730,-0.017090794,-0.005197031
Beta 2 - T-statistic,-0.5839792228,-0.838348316,-0.870885337,-1.961550620
Beta 2 - Alpha,-0.0109118196,-0.016061739,-0.016353293,-0.005441474
Beta 2 - T-statistic (alpha),-0.5648779184,-0.817357981,-0.855093591,-1.826449557
Beta 3 - Mean,-0.0007536626,-0.013515829,-0.016374830,-0.015621167


In [19]:
#Bivariate sort on size and skewness
bivariate_sort_size_skewness <- bivariate_sort(size, skewness_data)
bivariate_sort_size_skewness

,Size 1,Size 2,Size 3,Size 3-1
,<dbl>,<dbl>,<dbl>,<dbl>
Skewness 1 - Mean,-0.011062966,-0.0169392165,-0.0190303561,-0.007967390
Skewness 1 - T-statistic,-0.565985675,-0.8860890205,-0.9960068074,-2.681568114
Skewness 1 - Alpha,-0.007526473,-0.0148683108,-0.0154963629,-0.007969890
Skewness 1 - T-statistic (alpha),-0.398715553,-0.7704191040,-0.8189081915,-2.866074639
Skewness 2 - Mean,-0.008153785,-0.0172371110,-0.0167796058,-0.008625821
Skewness 2 - T-statistic,-0.419767418,-0.8194227714,-0.8066679705,-2.667218642
Skewness 2 - Alpha,-0.007389000,-0.0132201655,-0.0147716870,-0.007382687
Skewness 2 - T-statistic (alpha),-0.397052726,-0.6506661655,-0.7532089426,-2.357571277
Skewness 3 - Mean,-0.002490465,-0.0167912054,-0.0198752593,-0.017384794


### Fama-MacBeth regression

In [20]:
#Fama-Macbeth for skewness
FM_skewness <- fama_macbeth(list(Skewness = skewness_data))
FM_skewness

,(Intercept),Skewness
,<dbl>,<dbl>
Mean,-0.03588692,0.00192652
T-statistic,-1.87553272,1.08376316


In [21]:
#Fama-Macbeth for the three sort variables
indep_var <- list(Skewness = skewness_data, Size = size, Beta = betas)
FM_three_sorts <- fama_macbeth(indep_var)
FM_three_sorts

,(Intercept),Skewness,Size,Beta
,<dbl>,<dbl>,<dbl>,<dbl>
Mean,-0.01747055,-0.0006243586,-0.003488556,1.282459
T-statistic,-0.97320205,-0.7715815675,-4.127038566,1.783558


## 5. Conclusion

## Appendix A

### Data preparation

We performed the following steps to obtain the final data utilized in our analyses. Firstly, we loaded the data on Adjusted Close prices and Market capitalization which we collected in Homework 1. In addition, we downloaded the Fama-French 3 factors as well as the Momentum factor from [here](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html). The last required piece of data was Liquidity which we accessed at the [website](https://faculty.chicagobooth.edu/lubos-pastor/data) of Ľuboš Pástor.

In [ ]:
#Loading the data
yahoo_data <- readRDS("Raw data/list_yahoo.RData") #Adjusted close prices and volumes
market_cap <- readRDS("Raw data/market_cap_data.RData") #Market capitalization
fffactors_monthly <- read_csv("Raw data/F-F_Research_Data_Factors.CSV", skip = 2, n_max = 1149) #Need to limit the number of rows since the file contains yearly data as well
fffactors_daily <- read_csv("Raw data/F-F_Research_Data_Factors_daily.CSV", skip = 3, n_max = 25210) #Last row is some copyright => skip
momentum <- read_csv("Raw data/F-F_Momentum_Factor.CSV", skip = 12, n_max  = 1144, na = c("", "NA", -99.99, -999))
liquidity <- read.delim("Raw data/liq_data_1962_2021.txt", skip = 10, na = c("", "NA", -99.99, -999, -99))

Next, we define an auxilliary function which removes empty lines from an xts object (for future use).

In [ ]:
#Defining a function to strip an xts object of lines with 0 observations (all NAs)
strip_empty_lines <- function(xts_object) {
  na_sums <- apply(xts_object, 1, function(x) {sum(is.na(x))})
  indic <- na_sums < ncol(xts_object)
  return(xts_object[indic])
}

Then, we extract the Adjusted Close price.

In [ ]:
#Extracting the Adjusted Close Price
daily_adj_close <- lapply(yahoo_data, "[", , 6) #The unused argument is not an error, we have to skip it to specify columns

In order to calculate the excess returns, we need the market risk premium. Therefore, we extract it from the Fama-French factors data set. We also convert the daily observations to monthly and save the resulting data for future use in the portfolio analysis.

In [ ]:
#Extracting market risk premium and the risk-free rate
fffactors_monthly <- as.data.frame(fffactors_monthly) #Converting from tibble to a data frame
fffactors_daily <- as.data.frame(fffactors_daily) #Converting from tibble to a data frame
colnames(fffactors_daily)[1] <- "Date"
colnames(fffactors_monthly)[1] <- "Date"
fffactors_daily$Date <- as.Date(as.character(fffactors_daily$Date), "%Y%m%d") #Creating a date (daily)
fffactors_monthly$Date <-  as.Date(paste0(as.character(fffactors_monthly$Date),"01"), "%Y%m%d") + months(1) - days(1) #Creating a date (end of month)
market_prem_daily <- xts(fffactors_daily$`Mkt-RF`, order.by = fffactors_daily$Date)
names(market_prem_daily) <- "Mkt_RF_daily"
market_prem_monthly <- xts(fffactors_monthly$`Mkt-RF`, order.by = fffactors_monthly$Date)
names(market_prem_monthly) <- "Mkt_RF_monthly"
rf_daily <- xts(fffactors_daily$RF, order.by = fffactors_daily$Date)
names(rf_daily) <- "RF_daily"
rf_monthly <- xts(fffactors_monthly$RF, order.by = fffactors_monthly$Date)
names(rf_monthly) <- "RF_monthly"
fffactors_monthly_xts <- as.xts(fffactors_monthly[, 2:5], order.by = fffactors_monthly$Date) #Converting monthly factors to xts
fffactors_monthly_xts <- fffactors_monthly_xts["2007/"] #Restricting the period since we do not need older data
#saveRDS(fffactors_monthly_xts, file = "Final data/fffactors_monthly.RData") #Saving for future use

Now we can calculate the daily and monthly excess returns. The former will be used for the calculation of various measures such as beta and skewness. We save the latter for the portfolio analysis

In [ ]:
#Calculating excess returns
calc_excess_returns <- function(xts_object, daily = T) { #Defining a function to calculate excess returns
  if (daily == T) { #For daily returns
    result <- dailyReturn(xts_object)
    result <- result[2:nrow(result)] #Disregard the first observation (dailyReturn sets it to 0)
    risk_free_rate <- rf_daily[index(result)] #Define daily risk free rate
  } else { #For monthly returns
    result <- monthlyReturn(xts_object)
    risk_free_rate <- rf_monthly[index(result)]
  }
  return(result - risk_free_rate) #Substract risk free rate from the returns
}
#Daily excess returns
daily_returns <- lapply(daily_adj_close, calc_excess_returns) #Applying the function
daily_return_names <- gsub(".Adjusted", ".DailyReturns", unlist(lapply(daily_adj_close, names))) #New names
for (i in 1:length(daily_returns)) { #Renaming the columns for clarity
  names(daily_returns[[i]]) <- daily_return_names[i]
}
#Monthly excess returns
monthly_returns <- lapply(daily_adj_close, calc_excess_returns, daily = F)
monthly_return_names <- gsub(".Adjusted", ".MonthlyReturns", unlist(lapply(daily_adj_close, names))) #New names
for (i in 1:length(monthly_returns)) { #Renaming the columns for clarity
  names(monthly_returns[[i]]) <- monthly_return_names[i]
}
#Merge monthly excess returns
monthly_returns_merged <- do.call(merge.xts, monthly_returns)
#saveRDS(monthly_returns_merged, file = "Final data/monthly_excess_returns.RData") #Saving for future use

In the following step we define a function that calculates a specified monthly measure from past 12 months of daily data. In order to calculate the measure in a given period, we require at least 200 data points which is a standard for 12 months of daily data.

In [ ]:
#Defining a generic function to calculate a monthly measure from past 12 months of daily data
calc_measure <- function(xts_object, func, measure_name) { #Expects a series of monthly returns, the function to calculate the required measure (skewness, beta..) and the name of the measure
  result <- xts(rep(NA, nrow(xts_object)), order.by = index(xts_object)) #Empty xts object for the results
  names(result) <- gsub("MonthlyReturns", measure_name, names(xts_object)) #Naming the resulting object for clarity 
  ind <- which(monthly_return_names == names(xts_object)) #Get the rank of the stock in the list (1-250)
  for (i in 1:nrow(xts_object)) { #Loop through the rows of the object
    end_date <- index(xts_object)[i] #The last day that the daily returns will be used for computations
    start_date <- end_date %m-% months(12) #The start date is 12 months back as required
    df_to_use <- daily_returns[[ind]][paste0(start_date, "/", end_date)] #Extracting the subset we will use for the computation (12 months of daily returns)
    if (length(df_to_use) < 200){ #We require at least 200 data points, otherwise we won't calculate
      next
    } else {
      result[i] <- func(na.omit(df_to_use)) #Apply the specified function
    }
  }
  return(result)
}

We then apply the function on monthly returns for each of the 250 stocks. The first measure we calculate is Skewness (It is the third sorting variable that was assigned to our group). We save the result for future use.

In [ ]:
#Calculating skewness
skewness_data <- lapply(monthly_returns, calc_measure, func = skewness, measure_name = "Skewness") #Applying the function (skewness from the moments package)
#Merging skewness data
skewness_data_merged <- do.call(merge.xts, skewness_data)
skewness_data_merged <- strip_empty_lines(skewness_data_merged) #Remove empty lines from data (periods with no data)
#saveRDS(skewness_data_merged, file = "Final data/skewness.RData") #Saving for future use

Another sorting variable that we utilize in the portfolio analysis is Beta. We define a function that estimates beta for a given time series and then we pass it as an argument to the function above.

In [ ]:
#Calculating betas
calc_beta <- function(xts_object) { #Defining a function to calculate the beta (based on a series of daily returns)
  model_res <- lm(xts_object ~ market_prem_daily[index(xts_object)]) #Regress market premium on daily excess returns
  return(model_res$coefficients[[2]]) #Extract the estimated coefficient on market premium (beta)
}
#Applying the function
betas <- lapply(monthly_returns, calc_measure, func = calc_beta, measure_name = "Beta")
#Merging the betas
betas_merged <- do.call(merge.xts, betas)
betas_merged <- strip_empty_lines(betas_merged)
#saveRDS(betas_merged, file = "Final data/betas.RData") #Saving for future use

The last sorting variable we will use is Size. It can be calculated directly from Market Capitalization. We save the data for both of those factors.

In [ ]:
#Extracting monthly market cap data
market_cap_monthly <- lapply(market_cap, to.monthly, OHLC = F, indexAt = "lastof") #Converting daily to monthly
market_cap_monthly_merged <- do.call(merge.xts, market_cap_monthly) #Merging
market_cap_monthly_merged <- strip_empty_lines(market_cap_monthly_merged)
#saveRDS(market_cap_monthly_merged, file = "Final data/market_cap_monthly.RData") #Saving for future use

#Calculating size
size <- log(market_cap_monthly_merged)
names(size) <- gsub(".Adjusted", ".Size", unlist(lapply(daily_adj_close, names))) #New names
size <- as.xts(apply(size, 2, function(x) {ifelse(is.finite(x), x, NA)}), order.by = index(size)) #Replace -Inf with NA
size <- strip_empty_lines(size) #Remove empty rows
#saveRDS(size, file = "Final data/size.RData") #Saving for future use

We also extract the factors Momentum and Liquidity which are essential for the calculation of $\alpha$.

In [ ]:
#Extracting momentum
momentum <- as.data.frame(momentum)
colnames(momentum)[1] <- "Date"
momentum$Date <-  as.Date(paste0(as.character(momentum$Date[1]),"01"), "%Y%m%d") + months(1) - days(1) #Creating a date (end of month)
momentum_xts <- xts(momentum[, 2], order.by = momentum$Date) #Converting to xts
names(momentum_xts) <- "Momentum" #Renaming
momentum_xts <- momentum_xts["2007/"] #Restricting the period
#saveRDS(momentum_xts, "Final data/momentum.RData") #Saving for future use

#Extracting liquidity
colnames(liquidity)[1] <- "Date"
liquidity$Date <-  as.Date(paste0(as.character(liquidity$Date),"01"), "%Y%m%d") + months(1) - days(1) #Creating a date (end of month)
liquidity_xts <- xts(liquidity[, 4], order.by = liquidity$Date) #Converting to xts
names(liquidity_xts) <- "Liquidity" #Renaming
liquidity_xts <- liquidity_xts["2007/"] #Restricting the period
#saveRDS(liquidity_xts, "Final data/liquidity.RData") #Saving for future use

## Appendix B